# 📊 Notebook 01 - Exploration des Données

## Projet : Détection de Fraudes - Transactions par Chèque

**M2 SISE - Fouille de Données Massives**

---

### Objectifs de ce notebook

1. Charger et valider les données
2. Analyser les statistiques descriptives
3. Visualiser les distributions
4. Analyser le déséquilibre des classes
5. Identifier les corrélations et patterns

---

## 1. Configuration et imports

In [ ]:
# Imports standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Style des graphiques
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Taille par défaut des figures
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Imports réussis")

In [ ]:
# Import de notre module personnalisé
import sys
sys.path.append('..')

from config.config import (
    RAW_DATA_DIR, DATA_FILENAME, TARGET_COLUMN, 
    COLUMNS_TO_EXCLUDE, RANDOM_STATE, FIGURES_DIR
)
from src.data_loader import (
    load_raw_data, validate_columns, convert_data_types,
    get_data_summary, split_by_date, print_data_summary
)

print(f"📁 Dossier des données: {RAW_DATA_DIR}")
print(f"📁 Dossier des figures: {FIGURES_DIR}")

## 2. Chargement des données

In [ ]:
# Chargement des données brutes
# IMPORTANT: Modifiez DATA_FILENAME dans config/config.py si nécessaire

df = load_raw_data(verbose=True)

In [ ]:
# Aperçu des premières lignes
print("\n📋 Aperçu des données (5 premières lignes):")
df.head()

In [ ]:
# Aperçu des dernières lignes
print("\n📋 Aperçu des données (5 dernières lignes):")
df.tail()

In [ ]:
# Validation des colonnes
validation = validate_columns(df)

print("\n🔍 Validation des colonnes:")
print(f"   - Valide: {validation['valid']}")
print(f"   - Colonnes attendues: {validation['expected_count']}")
print(f"   - Colonnes trouvées: {validation['actual_count']}")

if validation['missing_columns']:
    print(f"   ⚠️ Colonnes manquantes: {validation['missing_columns']}")
if validation['extra_columns']:
    print(f"   ℹ️ Colonnes supplémentaires: {validation['extra_columns']}")

In [ ]:
# Conversion des types
df = convert_data_types(df, verbose=True)

## 3. Statistiques descriptives

In [ ]:
# Résumé global
summary = get_data_summary(df)
print_data_summary(summary)

In [ ]:
# Information sur les types de données
print("\n📊 Types de données:")
print(df.dtypes)

In [ ]:
# Statistiques descriptives pour les variables numériques
print("\n📈 Statistiques descriptives (variables numériques):")
df.describe().T

In [ ]:
# Valeurs manquantes détaillées
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Valeurs manquantes': missing,
    'Pourcentage (%)': missing_pct
}).sort_values('Valeurs manquantes', ascending=False)

print("\n❓ Valeurs manquantes par colonne:")
missing_df[missing_df['Valeurs manquantes'] > 0]

## 4. Analyse du déséquilibre des classes

In [ ]:
# Distribution de la variable cible
target_counts = df[TARGET_COLUMN].value_counts()
target_pct = df[TARGET_COLUMN].value_counts(normalize=True) * 100

print(f"\n🎯 Distribution de la variable cible '{TARGET_COLUMN}':")
print("\n   Valeur | Count      | Pourcentage")
print("   " + "-" * 40)
for val in target_counts.index:
    label = "Normal" if val == 0 else "Fraude"
    print(f"   {val} ({label:6s}) | {target_counts[val]:>10,} | {target_pct[val]:>6.2f}%")

imbalance_ratio = target_counts.min() / target_counts.max()
print(f"\n   📊 Ratio de déséquilibre: {imbalance_ratio:.4f} (1:{1/imbalance_ratio:.0f})")

In [ ]:
# Visualisation du déséquilibre
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique en barres
colors = ['#2ecc71', '#e74c3c']
bars = axes[0].bar(['Normal (0)', 'Fraude (1)'], target_counts.values, color=colors, edgecolor='black')
axes[0].set_ylabel('Nombre de transactions', fontsize=12)
axes[0].set_title('Distribution de la variable cible', fontsize=14, fontweight='bold')

# Ajouter les valeurs sur les barres
for bar, count, pct in zip(bars, target_counts.values, target_pct.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000, 
                 f'{count:,}\n({pct:.2f}%)', ha='center', va='bottom', fontsize=11)

# Graphique en camembert
axes[1].pie(target_counts.values, labels=['Normal (0)', 'Fraude (1)'], 
            autopct='%1.2f%%', colors=colors, explode=[0, 0.1],
            shadow=True, startangle=90)
axes[1].set_title('Proportion des classes', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n💾 Figure sauvegardée: {FIGURES_DIR / 'class_distribution.png'}")

## 5. Analyse des variables

In [ ]:
# Liste des variables numériques (excluant les identifiants et la date)
exclude_cols = ['ZIBZIN', 'IDAvisAutorisationCheque', 'DateTransaction', 'CodeDecision']
numeric_cols = [col for col in df.select_dtypes(include=[np.number]).columns 
                if col not in exclude_cols]

print(f"\n📊 Variables numériques à analyser ({len(numeric_cols)}):")
for i, col in enumerate(numeric_cols, 1):
    print(f"   {i:2d}. {col}")

In [ ]:
# Distribution du montant des transactions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogramme global
axes[0, 0].hist(df['Montant'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Montant (€)')
axes[0, 0].set_ylabel('Fréquence')
axes[0, 0].set_title('Distribution des montants (tous)')
axes[0, 0].axvline(df['Montant'].median(), color='red', linestyle='--', label=f"Médiane: {df['Montant'].median():.0f}€")
axes[0, 0].legend()

# Histogramme par classe
df[df[TARGET_COLUMN] == 0]['Montant'].hist(ax=axes[0, 1], bins=50, alpha=0.6, label='Normal', color='green')
df[df[TARGET_COLUMN] == 1]['Montant'].hist(ax=axes[0, 1], bins=50, alpha=0.6, label='Fraude', color='red')
axes[0, 1].set_xlabel('Montant (€)')
axes[0, 1].set_ylabel('Fréquence')
axes[0, 1].set_title('Distribution des montants par classe')
axes[0, 1].legend()

# Boxplot par classe
df.boxplot(column='Montant', by=TARGET_COLUMN, ax=axes[1, 0])
axes[1, 0].set_xlabel('Classe')
axes[1, 0].set_ylabel('Montant (€)')
axes[1, 0].set_title('Boxplot des montants par classe')
plt.suptitle('')

# Log-scale pour mieux voir
axes[1, 1].hist(df['Montant'], bins=50, edgecolor='black', alpha=0.7, log=True)
axes[1, 1].set_xlabel('Montant (€)')
axes[1, 1].set_ylabel('Fréquence (log)')
axes[1, 1].set_title('Distribution des montants (échelle log)')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'montant_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Statistiques du montant par classe
print("\n💰 Statistiques du montant par classe:")
df.groupby(TARGET_COLUMN)['Montant'].describe().T

In [ ]:
# Analyse de la variable CodeDecision
print("\n🔍 Distribution de CodeDecision:")
code_counts = df['CodeDecision'].value_counts().sort_index()
print(code_counts)

# Crosstab avec la cible
print("\n📊 Crosstab CodeDecision vs FlagImpaye:")
ct = pd.crosstab(df['CodeDecision'], df[TARGET_COLUMN], margins=True, margins_name='Total')
ct

In [ ]:
# Taux de fraude par CodeDecision
fraud_rate_by_code = df.groupby('CodeDecision')[TARGET_COLUMN].mean() * 100

print("\n📈 Taux de fraude par CodeDecision:")
for code, rate in fraud_rate_by_code.items():
    print(f"   Code {code}: {rate:.2f}%")

## 6. Analyse temporelle

In [ ]:
# Extraction des composantes temporelles
df['Date'] = df['DateTransaction'].dt.date
df['Month'] = df['DateTransaction'].dt.month
df['DayOfWeek'] = df['DateTransaction'].dt.dayofweek
df['Hour'] = df['DateTransaction'].dt.hour

# Transactions par mois
monthly_stats = df.groupby('Month').agg({
    TARGET_COLUMN: ['count', 'sum', 'mean']
}).round(4)
monthly_stats.columns = ['Total', 'Fraudes', 'Taux_fraude']

print("\n📅 Statistiques mensuelles:")
monthly_stats

In [ ]:
# Visualisation temporelle
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Transactions par mois
monthly_stats['Total'].plot(kind='bar', ax=axes[0, 0], color='steelblue', edgecolor='black')
axes[0, 0].set_xlabel('Mois')
axes[0, 0].set_ylabel('Nombre de transactions')
axes[0, 0].set_title('Transactions par mois')
axes[0, 0].tick_params(axis='x', rotation=0)

# Taux de fraude par mois
(monthly_stats['Taux_fraude'] * 100).plot(kind='bar', ax=axes[0, 1], color='coral', edgecolor='black')
axes[0, 1].set_xlabel('Mois')
axes[0, 1].set_ylabel('Taux de fraude (%)')
axes[0, 1].set_title('Taux de fraude par mois')
axes[0, 1].tick_params(axis='x', rotation=0)

# Transactions par jour de la semaine
dow_stats = df.groupby('DayOfWeek')[TARGET_COLUMN].agg(['count', 'mean'])
days = ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim']
axes[1, 0].bar(days, dow_stats['count'], color='steelblue', edgecolor='black')
axes[1, 0].set_xlabel('Jour de la semaine')
axes[1, 0].set_ylabel('Nombre de transactions')
axes[1, 0].set_title('Transactions par jour de la semaine')

# Taux de fraude par heure
hourly_fraud = df.groupby('Hour')[TARGET_COLUMN].mean() * 100
axes[1, 1].plot(hourly_fraud.index, hourly_fraud.values, marker='o', color='coral')
axes[1, 1].fill_between(hourly_fraud.index, hourly_fraud.values, alpha=0.3, color='coral')
axes[1, 1].set_xlabel('Heure')
axes[1, 1].set_ylabel('Taux de fraude (%)')
axes[1, 1].set_title('Taux de fraude par heure')
axes[1, 1].set_xlim(0, 23)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'temporal_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Analyse des corrélations

In [ ]:
# Matrice de corrélation
corr_cols = numeric_cols + [TARGET_COLUMN] if TARGET_COLUMN not in numeric_cols else numeric_cols
corr_matrix = df[corr_cols].corr()

# Heatmap
plt.figure(figsize=(16, 14))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
            fmt='.2f', square=True, linewidths=0.5,
            cbar_kws={'shrink': 0.8})
plt.title('Matrice de corrélation', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Corrélations avec la variable cible
target_corr = corr_matrix[TARGET_COLUMN].drop(TARGET_COLUMN).sort_values(key=abs, ascending=False)

print(f"\n🎯 Corrélations avec {TARGET_COLUMN} (triées par valeur absolue):")
print("\n   Variable               | Corrélation")
print("   " + "-" * 40)
for var, corr in target_corr.items():
    print(f"   {var:25s} | {corr:+.4f}")

In [ ]:
# Visualisation des corrélations avec la cible
fig, ax = plt.subplots(figsize=(12, 8))

colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in target_corr.values]
bars = ax.barh(target_corr.index, target_corr.values, color=colors, edgecolor='black')
ax.axvline(x=0, color='black', linewidth=0.8)
ax.set_xlabel('Corrélation avec FlagImpaye', fontsize=12)
ax.set_title('Corrélations des variables avec la variable cible', fontsize=14, fontweight='bold')

# Ajouter les valeurs
for bar, val in zip(bars, target_corr.values):
    x_pos = val + 0.01 if val >= 0 else val - 0.01
    ha = 'left' if val >= 0 else 'right'
    ax.text(x_pos, bar.get_y() + bar.get_height()/2, f'{val:.3f}', 
            va='center', ha=ha, fontsize=9)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'target_correlations.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Séparation Train/Test

In [ ]:
# Séparation temporelle selon le sujet
df_train, df_test = split_by_date(df, verbose=True)

In [ ]:
# Vérification de la cohérence
print("\n✅ Vérification de la séparation:")
print(f"   Train - Date min: {df_train['DateTransaction'].min()}")
print(f"   Train - Date max: {df_train['DateTransaction'].max()}")
print(f"   Test  - Date min: {df_test['DateTransaction'].min()}")
print(f"   Test  - Date max: {df_test['DateTransaction'].max()}")

# Vérifier qu'il n'y a pas de chevauchement
assert df_train['DateTransaction'].max() < df_test['DateTransaction'].min(), "⚠️ Chevauchement détecté!"
print("\n✅ Aucun chevauchement entre train et test")

## 9. Résumé et conclusions de l'exploration

In [ ]:
print("\n" + "=" * 70)
print("RÉSUMÉ DE L'EXPLORATION DES DONNÉES")
print("=" * 70)

print(f"\n📊 DONNÉES:")
print(f"   - Total: {len(df):,} transactions")
print(f"   - Variables: {len(df.columns)} colonnes")
print(f"   - Période: {df['DateTransaction'].min().date()} → {df['DateTransaction'].max().date()}")

print(f"\n🎯 DÉSÉQUILIBRE DES CLASSES:")
fraud_rate = df[TARGET_COLUMN].mean() * 100
print(f"   - Taux de fraude global: {fraud_rate:.2f}%")
print(f"   - Ratio: 1:{1/df[TARGET_COLUMN].mean():.0f}")
print(f"   → FORT DÉSÉQUILIBRE - nécessite techniques spécifiques")

print(f"\n📅 SÉPARATION TRAIN/TEST:")
print(f"   - Train: {len(df_train):,} ({len(df_train)/len(df)*100:.1f}%)")
print(f"   - Test: {len(df_test):,} ({len(df_test)/len(df)*100:.1f}%)")

print(f"\n🔗 VARIABLES LES PLUS CORRÉLÉES AVEC LA FRAUDE:")
for var, corr in target_corr.head(5).items():
    print(f"   - {var}: {corr:+.4f}")

print(f"\n⚠️ POINTS D'ATTENTION:")
print(f"   1. Ne PAS utiliser 'CodeDecision' pour la prédiction")
print(f"   2. Gérer le déséquilibre (SMOTE, class_weight, etc.)")
print(f"   3. Optimiser la F-mesure, pas l'accuracy")
print(f"   4. Respecter la séparation temporelle (pas de data leakage)")

print("\n" + "=" * 70)

In [ ]:
# Sauvegarde des données préparées pour le prochain notebook
from config.config import PROCESSED_DATA_DIR

df_train.to_pickle(PROCESSED_DATA_DIR / 'df_train.pkl')
df_test.to_pickle(PROCESSED_DATA_DIR / 'df_test.pkl')

print(f"\n💾 Données sauvegardées:")
print(f"   - {PROCESSED_DATA_DIR / 'df_train.pkl'}")
print(f"   - {PROCESSED_DATA_DIR / 'df_test.pkl'}")

---

## ➡️ Prochaine étape

Passez au notebook **02_preprocessing.ipynb** pour:
- Préparation des features
- Gestion des valeurs manquantes
- Encodage des variables
- Normalisation/Standardisation